In [ ]:
!pip install Pillow scipy numpy

In [ ]:
import numpy as np
from PIL import Image
from scipy.fftpack import dct, idct
from google.colab import files
import io
from skimage.metrics import structural_similarity as ssim

# Tải ảnh từ máy
def upload_image():
    uploaded = files.upload()
    for filename in uploaded.keys():
        return filename

# Đọc ảnh
def load_image(filename):
    image = Image.open(filename).convert('L') # Chuyển thành grayscale
    return np.array(image)

In [ ]:
# Chia ảnh thành các khối 8x8 và thêm padding
def split_into_blocks(image, block_size):
  height, width = image.shape
  pad_height = (block_size - height % block_size) % block_size
  pad_width = (block_size - width % block_size) % block_size
  padded_image = np.pad(image, ((0, pad_height), (0, pad_width)), mode='constant')

  height, width = padded_image.shape
  blocks = [padded_image[i:i+block_size, j:j+block_size] for i in range(0, height, block_size) for j in range(0, width, block_size)]
  return blocks, padded_image, (height-pad_height,width-pad_width)

In [ ]:
# Áp dụng DCT
def apply_dct(image):
    return dct(dct(image.T, norm='ortho').T, norm='ortho')

# Áp dụng IDCT
def apply_idct(image):
    return idct(idct(image.T, norm='ortho').T, norm='ortho')

In [ ]:
# Lượng tử hóa
def quantize(dct_block, quantization_matrix):
    return np.round(dct_block / quantization_matrix)

# Phân loại
def dequantize(quantized_block, quantization_matrix):
    return quantized_block * quantization_matrix

In [ ]:
# Mã hóa và giải mã (Huffman)
def encode_block(block):
  return block.flatten()

def decode_block(encoded_block, block_size):
  return encoded_block.reshape(block_size, block_size)

In [ ]:
# Ghép lại các khối thành ảnh
def combine_blocks(blocks, image_shape, block_size):
  height, width = image_shape
  image = np.zeros((height, width))
  block_idx = 0
  for i in range(0, height, block_size):
    for j in range(0, width, block_size):
      image[i:i+block_size, j:j+block_size] = blocks[block_idx]
      block_idx += 1
  return np.clip(image, 0, 255).astype(np.uint8)


In [ ]:
# Cắt bỏ padding
def remove_padding(image, original_shape):
  return image[:original_shape[0], :original_shape[1]]

In [ ]:
# Tính PSNR
def calculate_psnr(original_image, compressed_image):
  mse = np.mean((original_image - compressed_image) **2)
  if mse == 0:
    return float('inf')
  max_pixel = 255.0
  psnr = 20 * np.log10(max_pixel / np.sqrt(mse))
  return psnr
# Tính MSE
def calculate_mse(original_image, compressed_image):
  return np.mean((original_image - compressed_image) ** 2)
# Tính SSIM
def calculate_ssim(original_image, compressed_image):
  return ssim(original_image, compressed_image)

In [ ]:
# Tải ảnh từ máy lên Colab
filename = upload_image()
image = load_image(filename)

Saving E6TKzqA.jpg to E6TKzqA.jpg


In [ ]:
# Chia ảnh thành các khối 8x8
block_size = 8
blocks, padded_image, original_shape = split_into_blocks(image, block_size)
# Bảng lượng tử hóa đơn giản
quantization_matrix = np.ones((8,8))*10
# Nén ảnh
compressed_blocks = []
for block in blocks:
  dct_block = apply_dct(block)
  quantized_block = quantize(dct_block, quantization_matrix)
  encoded_block = encode_block(quantized_block)
  compressed_blocks.append(encoded_block)
# Giải nén ảnh
decompressed_blocks = []
for encoded_block in compressed_blocks:
  quantized_block = decode_block(encoded_block, block_size)
  dequantized_block = dequantize(quantized_block, quantization_matrix)
  idct_block = apply_idct(dequantized_block)
  decompressed_blocks.append(idct_block)
# Tạo ảnh từ các khối giải nén
decompressed_image = combine_blocks(decompressed_blocks, original_shape, 8)
# Cắt bỏ padding
decompressed_image = remove_padding(decompressed_image, original_shape)
# Lưu ảnh giải nén
output_path = "dec.jpg"
Image.fromarray(decompressed_image).save(output_path)

In [ ]:
# Tính thông số ảnh đầu vào
print("Thông số ảnh đầu vào:")
print("Kích thước ảnh:", image.shape)

Thông số ảnh đầu vào:
Kích thước ảnh: (1080, 1920)
